In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pymongo
import time
import concurrent.futures
from datetime import date, timedelta
import re

from requests_html import AsyncHTMLSession
import pyppdf.patch_pyppeteer

In [2]:
async def scrape_job_cards_dice(asession, search_string, location, start_page):
    
    #query URL
    url = f"""https://www.dice.com/jobs?q={search_string}&{location}&radius=30&radiusUnit=mi&page={start_page}&pageSize=5&language=en"""

    #get asynchronous request from search page
    r = await asession.get(url)
    
    # perform response rendering (running javascripts)
    print("rendering...")
    await r.html.arender()
    print("render finished")

    #parse request to BeautifulSoup object
    soup = bs(r.html.html, 'html.parser')

    #get page job cards
    return soup.find_all('div', class_="card")   

In [4]:
def scrape_job_dice(job_card):
    #initiate fields
    job_title = ""
    job_company = ""
    job_salary = ""
    job_location = ""
    job_date = ""
    job_description = ""

    job_title = job_card.find_all(class_="card-title-link")[0].text
    job_company = job_card.find_all(class_="card-company")[0].a.text

    # get location
    job_location = job_card.find_all(id="searchResultLocation")[0].text

    # get full job descr html
    job_descr_link = job_card.find_all(class_="card-title-link")[0].get('href')
    job_descr_html = requests.get(job_descr_link)
    soup = bs(job_descr_html.text, 'html.parser') 

    #check if salary is present or not
    try: 
        job_salary = soup.find_all(class_="mL20")[0].text
    except: 
        job_salary = ''

    # job description
    job_description = soup.find_all(id="jobdescSec")[0].get_text()

    # job date
    job_date = job_card.find_all(class_="posted-date")[0].text

    #print all found details

    print(f"""
        job_title: {job_title},
        job_company: {job_company},
        job_salary: {job_salary},
        job_location: {job_location},
        job_date: {job_date},
        job_description: {job_description[:30]}""")

    return [job_title, job_company, job_salary, job_location, job_date, job_description]


In [5]:
asession = AsyncHTMLSession()
search_string = "analyst"
location = "US"
start_page=1

In [6]:
# Store in db
class db_connection:
    def __init__(self):
        #connect to database
        connection_string='mongodb://localhost:27017'
        client = pymongo.MongoClient(connection_string)
        #define database for storage
        db = client.dice_db
        #drop all stored data
        db.dice_jobs.drop()
        db.dice_jobs
        #define collection to store data
        self.jobs_collection = db.jobs

    def store_job(self, title, company, salary, location, date, description):
        self.jobs_collection.insert_one({ \
        "job_title": title, \
        "job_company": company, \
        "job_salary": salary, \
        "job_location": location, \
        "job_date": date, \
        "job_description": description \
        })

In [7]:
#Initiate database session
session = db_connection()

In [ ]:
for start_page in [1,2,3,4,5]:
    # Scrape and store in DB
    cards = await scrape_job_cards_dice(asession, search_string, location, start_page)
    print(start_page)
    for card in cards:
        session.store_job(*scrape_job_dice(card))

rendering...
render finished
1

        job_title:  Analyst II, Information Technology Security ,
        job_company:  San Jacinto College ,
        job_salary: Depends on Experience,
        job_location:  Houston, TX, USA,
        job_date: 20 days ago,
        job_description: 
San Jacinto College Equal Opp

        job_title:  Data Analyst ,
        job_company:  Burning Glass Technologies ,
        job_salary: Depends on Experience,
        job_location:  Boston, MA, USA,
        job_date: 26 days ago,
        job_description: 
About Burning GlassAt Burning

        job_title:  DATA ANALYST ,
        job_company:  Collin Central Appraisal District ,
        job_salary: Depends on Experience,
        job_location:  McKinney, TX, USA,
        job_date: 59 days ago,
        job_description: 
POSITION: Data Analyst (521)D

        job_title:  Analyst ,
        job_company:  Genuent Global, LLC ,
        job_salary: BASED ON EXPERIENCE,
        job_location:  Glendale, CA, USA,
      

RuntimeError: This event loop is already running

In [ ]:
# Scrape and store in DB
cards = await scrape_job_cards_dice(asession, search_string, location, 1)
for card in cards:
    session.store_job(*scrape_job_dice(card))

In [ ]:
# Scrape and store in DB
cards = await scrape_job_cards_dice(asession, search_string, location, 2)
for card in cards:
    session.store_job(*scrape_job_dice(card))

In [ ]:
# # async method
# from requests_html import AsyncHTMLSession
# import pyppdf.patch_pyppeteer
# asession = AsyncHTMLSession()

In [ ]:
# #query URL
# url = f"""https://www.dice.com/jobs?q={search_string}&{location}&radius=30&radiusUnit=mi&page={start_page}&pageSize=20&language=en"""

In [ ]:
# # perform page request
# r = await asession.get(url)
# r.html.text

In [ ]:
# # perform rendering
# await r.html.arender()
# r.html.text

In [ ]:
# #get request from search page
# #r = requests_session.get(url)
# #r = requests.get(url)
# html = r.html.html

# #parse request to BeautifulSoup object
# soup = bs(html, 'html.parser')

# cards = soup.find_all('div', class_="card")

In [ ]:
# for card in cards:
#     # job title
#     print(card.find_all(class_="card-title-link")[0].text)
#     # job descr link
#     job_descr_link = card.find_all(class_="card-title-link")[0].get('href')
#     print(job_descr_link)
#     # job salary
#     job_descr_html = requests.get(job_descr_link)
#     soup = bs(job_descr_html.text, 'html.parser')  
#     try: 
#         job_salary = soup.find_all(class_="mL20")[0].text
#     except: 
#         job_salary = ''
#     print(job_salary)
#     # job description
#     job_description = soup.find_all(id="jobdescSec")[0].get_text()
#     print(job_description)
#     # job company
#     print(card.find_all(class_="card-company")[0].a.text)
#     # job location
#     print(card.find_all(id="searchResultLocation")[0].text)
#     # job type
#     print(card.find_all(class_="card-position-type")[0].text)
#     # posted date
#     job_date = card.find_all(class_="posted-date")[0].text